In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [2]:
test_articles = pd.read_csv('../dataset/test_articles_ourdata.csv', delimiter=',')

In [5]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_ourdata.csv', delimiter=',')

In [6]:
embedding_model = Doc2Vec.load('../pre_trained_models/doc2vec.bin')

In [7]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [8]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [9]:
test_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   article_id                948 non-null    int64 
 1   article_title             948 non-null    object
 2   article_full_text         948 non-null    object
 3   article_meta_description  948 non-null    object
 4   article_summary           948 non-null    object
 5   article_keywords          948 non-null    object
 6   article_meta_key_words    662 non-null    object
 7   article_tags              207 non-null    object
dtypes: int64(1), object(7)
memory usage: 59.4+ KB


In [10]:
fixed_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94721 entries, 0 to 94720
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label_index              94721 non-null  int64 
 1   table_page_id            94721 non-null  int64 
 2   table_page_title         94721 non-null  object
 3   table_page_main_passage  94721 non-null  object
 4   table_page_keywords      94721 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


In [11]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_id']
        article_title_text = str(row['article_title'])
        article_meta_description_text = str(row['article_meta_description'])
        article_keywords_text = str(row['article_keywords'])

        #embedding and model variables
        article_title = []
        title_table = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_id']]
        
        vector_words = tknzr.tokenize(article_title_text+" "+article_meta_description_text+" "+article_keywords_text)
        word_embedding = embedding_model.infer_vector(vector_words)
        article_title.append(word_embedding)

        #creating embedding 
        for i, row in return_index.iterrows():
            
            vector_words = tknzr.tokenize(str(row['table_page_title'])+" "+str(row['table_page_main_passage'])+" "+str(row['table_page_keywords']))
            word_embedding = embedding_model.infer_vector(vector_words)
            title_table.append(word_embedding)
        
        distance_vector = pairwise_distances(article_title, title_table, metric='cosine')
        
        #creating the final dataframe
        for i in range(0,len(distance_vector[0])):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],distance_vector[0][i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking')
        
        selected_top = data_frame_sorting.head(TOP_K)
#         max_score = selected_top['table_ranking'].max()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] <= max_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")

948it [06:53,  2.29it/s]
948it [06:37,  2.38it/s]
948it [06:38,  2.38it/s]
948it [06:36,  2.39it/s]
948it [06:39,  2.37it/s]


['Acc@1', '0.3365']
['Acc@5', '0.6297']
['Acc@10', '0.7563']
['Acc@20', '0.8713']
['Acc@50', '0.9715']
MRR: 0.4611
[0.3333333333333333, 0.16666666666666666, 1.0, 0.125, 0.3333333333333333, 0.5, 0.5, 0.5, 1.0, 0.16666666666666666, 0.05555555555555555, 0.16666666666666666, 0.038461538461538464, 1.0, 0.2, 0.16666666666666666, 0.16666666666666666, 0.25, 0, 1.0, 1.0, 0.1111111111111111, 1.0, 0.02564102564102564, 0.06666666666666667, 0.16666666666666666, 0.25, 0.3333333333333333, 0.07142857142857142, 0.5, 0.5, 1.0, 0.3333333333333333, 0.043478260869565216, 1.0, 0.06666666666666667, 0.07692307692307693, 1.0, 0.08333333333333333, 1.0, 1.0, 1.0, 0.022222222222222223, 0.1, 0.045454545454545456, 0.05263157894736842, 0.08333333333333333, 0.16666666666666666, 0.043478260869565216, 0.2, 0, 1.0, 0.09090909090909091, 0.07142857142857142, 0.1111111111111111, 1.0, 1.0, 1.0, 1.0, 0.06666666666666667, 0.2, 1.0, 0, 1.0, 0.16666666666666666, 0.027777777777777776, 1.0, 0.3333333333333333, 0.0909090909090909